# Тюнинг гиперпараметров моделей (Optuna) для trading_predictor

Этот ноутбук запускает подбор гиперпараметров для моделей XGBoost/LightGBM/CatBoost с использованием TimeSeriesSplit и сохраняет результаты в `06_reports/exp_<id>_<ts>/`.

- Целевая метрика: F1 (бинарная цель `y_bs` — 0/1).
- Структура результатов повторяет подход `modeling_walkthrough.ipynb`.
- Лучшие параметры автоматически переносятся в `04_configs/models.yml` (с созданием резервной копии).


In [1]:
# Настройка окружения и путей проекта
from pathlib import Path
from datetime import datetime
import os
import yaml
import numpy as np
import pandas as pd

# Поиск корня проекта (как в modeling_walkthrough)
def _find_project_root(start: Path) -> Path:
    base = start if start.is_dir() else start.parent
    for candidate in [base] + list(base.parents):
        if (candidate / "03_src").exists() and (candidate / "04_configs").exists():
            return candidate
    raise RuntimeError(
        "Не удалось найти корень проекта. Убедитесь, что существуют папки '03_src' и '04_configs' выше по дереву."
    )

try:
    _here = Path(__file__).resolve()
except NameError:
    _here = Path.cwd().resolve()

PROJECT_ROOT = _find_project_root(_here)
DATA_PROCESSED = PROJECT_ROOT / "01_data/processed"
REPORTS_DIR = PROJECT_ROOT / "06_reports"
CONFIG_SPLITS = PROJECT_ROOT / "04_configs/splits.yml"
CONFIG_MODELS = PROJECT_ROOT / "04_configs/models.yml"
CONFIG_HPT = PROJECT_ROOT / "04_configs/hyperparameter_tuning.yml"

REPORTS_DIR.mkdir(parents=True, exist_ok=True)

# Определяем имя каталога запуска 06_reports/exp_<id>_<ts>
try:
    with open(CONFIG_MODELS, "r", encoding="utf-8") as f:
        _models_cfg = yaml.safe_load(f) or {}
        _exp_id_cfg = ((_models_cfg.get("experiment") or {}).get("current_id") or "exp_0000")
except Exception:
    _exp_id_cfg = "exp_0000"

RUN_DIR = REPORTS_DIR / f"{str(_exp_id_cfg).lower()}_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
RUN_DIR.mkdir(parents=True, exist_ok=True)
REPORTS_DIR = RUN_DIR

print(f"Каталог результатов текущего запуска: {REPORTS_DIR}")


Каталог результатов текущего запуска: C:\Users\Shchurov\Хранилище\Документы\05_Программирование\02_projects\personal\trading_predictor\06_reports\exp_0018_20251021_121253


In [2]:
# Определение пути к файлу признаков по feature_engineering.yml
cfg_path = PROJECT_ROOT / "04_configs" / "feature_engineering.yml"
try:
    with open(cfg_path, "r", encoding="utf-8") as f:
        fe_cfg = yaml.safe_load(f) or {}
except FileNotFoundError:
    raise FileNotFoundError(
        f"Не найден конфиг Feature Engineering: {cfg_path}. Проверьте путь '04_configs/feature_engineering.yml'."
    )

ps = (fe_cfg.get("pipeline_settings") or {})
feature_set = ps.get("feature_set")
if not feature_set:
    raise KeyError(
        "В конфиге отсутствует 'pipeline_settings.feature_set'. Укажите активный набор признаков (например, 'fset-1')."
    )

fsets = (fe_cfg.get("feature_sets") or {})
if feature_set not in fsets:
    raise KeyError(
        f"Указанный набор признаков '{feature_set}' отсутствует в секции 'feature_sets' файла конфига."
    )

ds = (fe_cfg.get("dataset") or {})
active = ds.get("active")
items = (ds.get("items") or {})
symbol = (items.get(active) or {}).get("symbol", "ASSET")
symbol_lower = str(symbol).lower()

output_dir = ps.get("output_dir", "01_data/processed")
output_pattern = ps.get("output_pattern", "{symbol_lower}_{feature_set}_features.parquet")

FEATURES_FILE = PROJECT_ROOT / output_dir / output_pattern.format(
    symbol_lower=symbol_lower,
    feature_set=feature_set,
)

if not FEATURES_FILE.exists():
    raise FileNotFoundError(
        f"Не найден файл фич: {FEATURES_FILE}. Сначала запустите пайплайн feature engineering."
    )

print("Активный датасет:", active)
print("Набор признаков:", feature_set)
print("Файл признаков:", FEATURES_FILE)


Активный датасет: eurusd_h1_oanda_2010_2024
Набор признаков: fset-15
Файл признаков: C:\Users\Shchurov\Хранилище\Документы\05_Программирование\02_projects\personal\trading_predictor\01_data\processed\eurusd_fset-15_features.parquet


In [3]:
# Запуск тюнинга гиперпараметров
from importlib import import_module

# Делаем импорт модуля проекта
import sys
sys.path.append(str(PROJECT_ROOT / "03_src"))

from models.hyperparameter_tuning import tune_hyperparameters # type: ignore[reportMissingImports]

results = tune_hyperparameters(
    models=None,  # берём все enabled из models.yml
    n_trials=None,  # возьмём из hyperparameter_tuning.yml
    config_path=str(CONFIG_HPT.relative_to(PROJECT_ROOT)),
    features_path=str(FEATURES_FILE.relative_to(PROJECT_ROOT)),
    splits_config_path=str(CONFIG_SPLITS.relative_to(PROJECT_ROOT)),
    models_config_path=str(CONFIG_MODELS.relative_to(PROJECT_ROOT)),
    reports_dir=str(REPORTS_DIR.relative_to(PROJECT_ROOT)),
    run_dir=str(REPORTS_DIR),
    update_models_yaml=True,
)

print("Готово. Результаты сохранены в:", results["run_dir"])


[I 2025-10-21 12:12:58,536] A new study created in memory with name: no-name-928db5cd-e95d-4aae-ab36-437bebd013c1
[I 2025-10-21 12:13:01,213] Trial 0 finished with value: 0.45749461298722094 and parameters: {'learning_rate': 0.013364184785606975, 'n_estimators': 422, 'max_depth': 8, 'min_child_weight': 7, 'subsample': 0.8694940652038774, 'colsample_bytree': 0.5797415090497895, 'reg_lambda': 0.32323205495774304, 'reg_alpha': 2.57742734610013, 'gamma': 1.8150584871125608}. Best is trial 0 with value: 0.45749461298722094.
[I 2025-10-21 12:13:02,983] Trial 1 finished with value: 0.4104817249372047 and parameters: {'learning_rate': 0.02377062752207792, 'n_estimators': 1998, 'max_depth': 8, 'min_child_weight': 5, 'subsample': 0.5877217961200225, 'colsample_bytree': 0.9264200732616141, 'reg_lambda': 0.4127801205343093, 'reg_alpha': 0.003155662246020448, 'gamma': 3.0260478722094994}. Best is trial 0 with value: 0.45749461298722094.
[I 2025-10-21 12:13:04,442] Trial 2 finished with value: 0.450

Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[327]	valid_0's binary_logloss: 0.693766
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[782]	valid_0's binary_logloss: 0.691774
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[189]	valid_0's binary_logloss: 0.693069
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[872]	valid_0's binary_logloss: 0.692804
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[1088]	valid_0's binary_logloss: 0.692336


[I 2025-10-21 12:15:09,911] Trial 0 finished with value: 0.37174502099661133 and parameters: {'learning_rate': 0.0010344694260839397, 'n_estimators': 4236, 'num_leaves': 218, 'max_depth': 6, 'feature_fraction': 0.6537655796177148, 'bagging_fraction': 0.7316027360734805, 'bagging_freq': 1, 'min_data_in_leaf': 195, 'lambda_l1': 0.0020886861520553633, 'lambda_l2': 0.01480388413261023, 'min_split_gain': 0.8126000587970541}. Best is trial 0 with value: 0.37174502099661133.


Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[5]	valid_0's binary_logloss: 0.693918
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[35]	valid_0's binary_logloss: 0.691612
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[8]	valid_0's binary_logloss: 0.693139
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[58]	valid_0's binary_logloss: 0.692734
Training until validation scores don't improve for 50 rounds


[I 2025-10-21 12:15:11,969] Trial 1 finished with value: 0.3383122504876533 and parameters: {'learning_rate': 0.013468774732492785, 'n_estimators': 3070, 'num_leaves': 63, 'max_depth': 8, 'feature_fraction': 0.6749345684332166, 'bagging_fraction': 0.6515348647920541, 'bagging_freq': 2, 'min_data_in_leaf': 19, 'lambda_l1': 0.001191748287167473, 'lambda_l2': 2.767908519093944, 'min_split_gain': 0.8780574327295113}. Best is trial 0 with value: 0.37174502099661133.


...

Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[198]	valid_0's binary_logloss: 0.693695
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[791]	valid_0's binary_logloss: 0.691241
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[200]	valid_0's binary_logloss: 0.693033
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[604]	valid_0's binary_logloss: 0.692729
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[787]	valid_0's binary_logloss: 0.692306


[I 2025-10-21 12:18:06,767] Trial 49 finished with value: 0.42672064958583594 and parameters: {'learning_rate': 0.0015369695016075246, 'n_estimators': 1914, 'num_leaves': 53, 'max_depth': 5, 'feature_fraction': 0.6518725467291512, 'bagging_fraction': 0.7059577002098263, 'bagging_freq': 2, 'min_data_in_leaf': 10, 'lambda_l1': 0.0011439357170150925, 'lambda_l2': 0.26362313382027736, 'min_split_gain': 0.51235424188159}. Best is trial 47 with value: 0.4567273085782205.


Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[779]	valid_0's binary_logloss: 0.692375


c:\Users\Shchurov\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_plot\roc_curve.py:171: UserWarning: No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
  self.ax_.legend(loc="lower right")
[I 2025-10-21 12:18:09,645] A new study created in memory with name: no-name-27ab61d7-d759-408a-8d33-261f68de016a
[I 2025-10-21 12:18:36,694] Trial 0 finished with value: 0.5482618284999923 and parameters: {'learning_rate': 0.005541890104257469, 'iterations': 2138, 'depth': 9, 'l2_leaf_reg': 9.68591893090139, 'random_strength': 0.6895117246747574, 'bagging_temperature': 0.25911341702591495}. Best is trial 0 with value: 0.5482618284999923.
[I 2025-10-21 12:18:40,516] Trial 1 finished with value: 0.5503706903316052 and parameters: {'learning_rate': 0.08694531759817549, 'iterations': 4067, 'depth': 6, 'l2_leaf_reg': 2.6079473160453865, 'random_strength': 0.710129958813901

Готово. Результаты сохранены в: C:\Users\Shchurov\Хранилище\Документы\05_Программирование\02_projects\personal\trading_predictor\06_reports\exp_0018_20251021_121253


In [4]:
# Вывод сводной таблицы и ключевых метрик
import json
from pprint import pprint

summary_csv = REPORTS_DIR / "hyperparameter_tuning_summary.csv"
results_json = REPORTS_DIR / "hyperparameter_tuning_results.json"

if summary_csv.exists():
    display(pd.read_csv(summary_csv))
else:
    print("Не найден файл:", summary_csv)

if results_json.exists():
    with open(results_json, "r", encoding="utf-8") as f:
        res = json.load(f)
    print("\nЛучшие параметры по моделям:")
    for m, data in res.items():
        print(f"\n— {m}")
        pprint(data.get("best_params", {}))
else:
    print("Не найден файл:", results_json)


,model,cv_primary,cv_primary_best,test_f1,test_balanced_accuracy,test_accuracy,test_logloss,test_roc_auc,n_trials,best_params
0,xgboost,f1,0.511986,0.607107,0.503148,0.502787,0.693158,0.510620,50,"{""learning_rate"": 0.2947909664772785, ""n_estim..."
1,lightgbm,f1,0.456727,0.623744,0.515307,0.514914,0.692375,0.526240,50,"{""learning_rate"": 0.0014214922476224001, ""n_es..."
2,catboost,f1,0.569918,0.613550,0.519540,0.519208,0.692023,0.528939,50,"{""learning_rate"": 0.015457211061693104, ""itera..."



Лучшие параметры по моделям:

— xgboost
{'colsample_bytree': 0.5230042839579707,
 'gamma': 0.9803317817084722,
 'learning_rate': 0.2947909664772785,
 'max_depth': 9,
 'min_child_weight': 7,
 'n_estimators': 1291,
 'reg_alpha': 1.6169450051501335,
 'reg_lambda': 0.16346259914233485,
 'subsample': 0.8421491510163616}

— lightgbm
{'bagging_fraction': 0.7354008932141739,
 'bagging_freq': 1,
 'feature_fraction': 0.6478981365112846,
 'lambda_l1': 0.0015845784175926116,
 'lambda_l2': 0.03405533820825587,
 'learning_rate': 0.0014214922476224001,
 'max_depth': 5,
 'min_data_in_leaf': 28,
 'min_split_gain': 0.5477693244460864,
 'n_estimators': 2067,
 'num_leaves': 100}

— catboost
{'bagging_temperature': 0.4110687110789265,
 'depth': 5,
 'iterations': 3729,
 'l2_leaf_reg': 7.99892109706654,
 'learning_rate': 0.015457211061693104,
 'random_strength': 0.4223956034287203}


## Примечания

- Все артефакты текущего запуска сохранены в каталоге `06_reports/exp_<id>_<ts>/`.
- Файл `best_params.yml` содержит лучшие параметры по моделям; они уже применены к `04_configs/models.yml` с созданием резервной копии.
- Матрицы ошибок и ROC‑кривые сохраняются как `cm_<model>.png` и `roc_<model>.png` (если доступны вероятности).


### Автологирование результатов тюнинга в experiments.csv

Эта ячейка записывает строку в `06_reports/experiments.csv` с метриками теста и лучшими гиперпараметрами выбранной модели (по `primary` из `04_configs/models.yml`, fallback — `f1`).

Выполните её после завершения тюнинга и формирования файлов `hyperparameter_tuning_summary.csv` и `hyperparameter_tuning_results.json`.


In [5]:
# Автологирование эксперимента (тюнинг гиперпараметров) в 06_reports/experiments.csv
from pathlib import Path
from datetime import datetime, timezone
import json
import yaml
import numpy as np
import pandas as pd

# Путь к сводному журналу экспериментов
EXPERIMENTS_CSV = PROJECT_ROOT / "06_reports" / "experiments.csv"

# Считываем primary-метрику и ID эксперимента из models.yml
PRIMARY_DEFAULT = None
CURRENT_EXP_ID = None
try:
    with open(CONFIG_MODELS, "r", encoding="utf-8") as f:
        _models_cfg = yaml.safe_load(f) or {}
        PRIMARY_DEFAULT = _models_cfg.get("metrics", {}).get("primary")
        CURRENT_EXP_ID = (_models_cfg.get("experiment") or {}).get("current_id")
except Exception:
    PRIMARY_DEFAULT = None
    CURRENT_EXP_ID = None


def _infer_asset_tf(fe_cfg: dict) -> str:
    # 1) dataset.items[active]
    try:
        ds = (fe_cfg.get("dataset") or {})
        active_key = ds.get("active")
        item = (ds.get("items") or {}).get(active_key) or {}
        symbol = item.get("symbol")
        tf = item.get("timeframe")
        if symbol and tf:
            return f"{str(symbol).upper()}/{str(tf).upper()}"
    except Exception:
        pass
    # 2) eurusd_h1_... -> EURUSD/H1
    try:
        active_key = ((fe_cfg.get("dataset") or {}).get("active") or "")
        parts = str(active_key).split("_")
        if len(parts) >= 2:
            asset = parts[0]
            tf = parts[1]
            if asset and tf:
                return f"{asset.upper()}/{tf.upper()}"
    except Exception:
        pass
    # 3) Фоллбек по имени входного файла
    try:
        input_file = fe_cfg.get("pipeline_settings", {}).get("input_file", "")
        base = Path(input_file).name
        parts = base.split("_")
        if len(parts) >= 3:
            asset = parts[0]
            tf = parts[2]
            return f"{asset}/{tf}"
    except Exception:
        pass
    return "UNKNOWN/TF"


def _infer_ids(fe_cfg: dict) -> tuple[str, str, str]:
    # Dataset — из feature_engineering.yml → dataset.active
    dataset_id = str(((fe_cfg.get("dataset") or {}).get("active")) or "unknown_dataset")

    # Target — пробуем из feature_definitions.y_bs, иначе из верхнего уровня y_bs
    y_node = None
    if isinstance(fe_cfg.get("feature_definitions"), dict):
        y_node = (fe_cfg.get("feature_definitions") or {}).get("y_bs")
    if not y_node and isinstance(fe_cfg.get("y_bs"), dict):
        y_node = fe_cfg.get("y_bs")
    params = (y_node.get("params") if isinstance(y_node, dict) else {}) or {}
    horizon = params.get("horizon")
    method = (y_node.get("method") if isinstance(y_node, dict) else "binary_target")
    h_part = f"h{int(horizon)}" if isinstance(horizon, (int, float)) else "h?"
    m_part = "binary" if "binary" in str(method).lower() else str(method).lower()
    target_id = f"target_{h_part}_{m_part}"

    # Feature set — из pipeline_settings.feature_set
    ps = (fe_cfg.get("pipeline_settings") or {})
    fset_id = str(ps.get("feature_set") or "fset-?")

    return dataset_id, target_id, fset_id


def _round3(x: float | None) -> float | None:
    if x is None or (isinstance(x, float) and (np.isnan(x) or np.isinf(x))):
        return None
    return float(np.round(x, 3))


# 1) Читаем артефакты тюнинга
summary_csv = REPORTS_DIR / "hyperparameter_tuning_summary.csv"
results_json_fp = REPORTS_DIR / "hyperparameter_tuning_results.json"
if not summary_csv.exists() or not results_json_fp.exists():
    raise FileNotFoundError("Не найдены файлы результатов тюнинга. Сначала выполните ячейку запуска Optuna.")

summary_df = pd.read_csv(summary_csv)
with open(results_json_fp, "r", encoding="utf-8") as f:
    hpt_results = json.load(f)

# 2) Выбираем модель по primary (из models.yml) — по test_<metric>, иначе fallback на cv_primary_best
metric = (PRIMARY_DEFAULT or "f1").strip().lower()
sort_col = f"test_{metric}"
if sort_col not in summary_df.columns or summary_df[sort_col].isna().all():
    sort_col = "cv_primary_best" if "cv_primary_best" in summary_df.columns else summary_df.columns[0]
summary_sorted = summary_df.sort_values(by=[sort_col], ascending=False)
chosen_model = str(summary_sorted.iloc[0]["model"]) if not summary_sorted.empty else None

# 3) Формируем строку для experiments.csv
with open(PROJECT_ROOT / "04_configs" / "feature_engineering.yml", "r", encoding="utf-8") as f:
    fe_cfg = yaml.safe_load(f) or {}
with open(CONFIG_SPLITS, "r", encoding="utf-8") as f:
    splits_cfg = yaml.safe_load(f) or {}

# Метрики и детали по выбранной модели берём из результатов тюнинга
metrics_map = {m: None for m in [
    "accuracy", "f1", "precision", "recall", "balanced_accuracy", "roc_auc"
]}
up_sh, down_sh = None, None
if chosen_model and chosen_model in hpt_results:
    payload = hpt_results[chosen_model]
    test_metrics = payload.get("test_metrics", {})
    test_details = payload.get("test_details", {})
    metrics_map["accuracy"] = test_metrics.get("accuracy")
    metrics_map["f1"] = test_metrics.get("f1")
    metrics_map["balanced_accuracy"] = test_metrics.get("balanced_accuracy")
    metrics_map["roc_auc"] = test_metrics.get("roc_auc")

    cls_report = (test_details.get("classification_report") or {})
    weighted = cls_report.get("weighted avg") or {}
    if "precision" in weighted:
        metrics_map["precision"] = weighted.get("precision")
    if "recall" in weighted:
        metrics_map["recall"] = weighted.get("recall")

    # Доли классов
    try:
        labels = ["0", "1"]
        supports = [cls_report.get(lbl, {}).get("support") for lbl in labels]
        if all(isinstance(s, (int, float)) for s in supports):
            total = float(sum(supports)) if sum(supports) else 1.0
            down_sh = float(supports[0]) / total
            up_sh = float(supports[1]) / total
    except Exception:
        pass
    if up_sh is None or down_sh is None:
        cm = np.array(test_details.get("confusion_matrix") or [])
        if cm.size > 0:
            supports = cm.sum(axis=1)
            total = float(supports.sum()) if supports.sum() else 1.0
            down_sh = float(supports[0]) / total
            up_sh = float(supports[1]) / total

# Округляем
acc = _round3(metrics_map["accuracy"]) if metrics_map["accuracy"] is not None else None
f1 = _round3(metrics_map["f1"]) if metrics_map["f1"] is not None else None
precision = _round3(metrics_map["precision"]) if metrics_map["precision"] is not None else None
recall = _round3(metrics_map["recall"]) if metrics_map["recall"] is not None else None
bal_acc = _round3(metrics_map["balanced_accuracy"]) if metrics_map["balanced_accuracy"] is not None else None
roc_auc = _round3(metrics_map["roc_auc"]) if metrics_map["roc_auc"] is not None else None

# Asset/TF, Dataset/Target/Feature set
asset_tf = _infer_asset_tf(fe_cfg)
dataset_id, target_id, fset_id = _infer_ids(fe_cfg)

# Primary-метрика и её значение
primary_metric = metric
primary_value = None
if metric in {"f1", "balanced_accuracy", "accuracy", "roc_auc"}:
    primary_value = {"f1": f1, "balanced_accuracy": bal_acc, "accuracy": acc, "roc_auc": roc_auc}[metric]

# Валидация (короткая строка вида tscv_k=5_gap=5)
val_scheme = splits_cfg.get("time_series_cv", {})
n_splits = val_scheme.get("n_splits")
gap = val_scheme.get("gap")
validation_str = f"tscv_k={n_splits}" if n_splits else "tscv"
if gap:
    validation_str += f"_gap={gap}"

# Params — лучшие гиперпараметры выбранной модели
params_for_row = ""
if chosen_model and chosen_model in hpt_results:
    params_for_row = json.dumps(hpt_results[chosen_model].get("best_params", {}), ensure_ascii=False)

# Собираем строку и пишем в CSV
columns = [
    "ID","Date","Asset/TF","Dataset","Target","Feature set","Model","Params",
    "Validation","Seed","Acc","F1","Precision","Recall","BalancedAcc",
    "ROC-AUC","Up %","Down %","Primary metric","Primary value",
]
now_utc = datetime.now(timezone.utc).isoformat(timespec="seconds")
row = [
    str(CURRENT_EXP_ID or "exp_0000").upper(),
    now_utc,
    asset_tf,
    dataset_id,
    target_id,
    fset_id,
    chosen_model or "",
    params_for_row,
    validation_str,
    42,
    acc,
    f1,
    precision,
    recall,
    bal_acc,
    roc_auc,
    _round3(up_sh) if up_sh is not None else None,
    _round3(down_sh) if down_sh is not None else None,
    primary_metric,
    _round3(primary_value) if primary_value is not None else None,
]

EXPERIMENTS_CSV.parent.mkdir(parents=True, exist_ok=True)
if EXPERIMENTS_CSV.exists():
    df_csv = pd.read_csv(EXPERIMENTS_CSV)
    if set(df_csv.columns) != set(columns):
        df_csv = df_csv.reindex(columns=columns)
    df_csv = pd.concat([df_csv, pd.DataFrame([row], columns=columns)], ignore_index=True)
else:
    df_csv = pd.DataFrame([row], columns=columns)

df_csv.to_csv(EXPERIMENTS_CSV, index=False)
print(f"Запись эксперимента добавлена: {EXPERIMENTS_CSV}")


Запись эксперимента добавлена: C:\Users\Shchurov\Хранилище\Документы\05_Программирование\02_projects\personal\trading_predictor\06_reports\experiments.csv
